# Dataflow code for cloud postgresql (read data from table and load into table)
# Able to load data from jupyter without creating dataflow instance.
# Able to create dataflow instance but failing with below error
# (ModuleNotFoundError: No module named 'sqlalchemy.cimmutabledict')

In [ ]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import SetupOptions
from beam_nuggets.io import relational_db
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners import DataflowRunner
import google.auth

In [ ]:
#with beam.Pipeline(InteractiveRunner(), options=PipelineOptions()) as p:
#    source_config = relational_db.SourceConfiguration(
#        drivername='postgresql+pg8000',
#        host='127.0.0.1',
#        port=5432,
#        username='postgres',
#        password='newpass1',
#        database='mart',
#        )
#    records = p | "Reading records from db" >> relational_db.ReadFromDB(
#        source_config=source_config,        
#        table_name='item_details_p',
#        #query='select count(1) from item_details_p'
#        )
#    records | 'Writing to stdout' >> beam.Map(print)

In [33]:
p=beam.Pipeline(InteractiveRunner(), options=PipelineOptions())

source_config = relational_db.SourceConfiguration(
        drivername='postgresql+pg8000',
        host='127.0.0.1',
        port=5432,
        username='postgres',
        password='newpass1',
        database='mart',
        create_if_missing=True,
        )
# Target database table
table_config = relational_db.TableConfiguration(
        name='item_details_pw',
        create_if_missing=True,  # automatically create the table if not there
        primary_key_columns=['item_identifier']  # and use 'num' column as a primary key
    )

records = p | "Reading records from db" >> relational_db.ReadFromDB(
        source_config=source_config,        
        table_name='item_details_p',
        )

#records = p | "Reading records" >> beam.Create([
#        {'item_identifier': 'NCD19', 'item_weight': 8.93, 'item_fat_content': 'Low Fat'},
#        {'item_identifier': 'FDX07', 'item_weight': 19.2, 'item_fat_content': 'Regular'},
#    ])

In [3]:
ib.show(records)

In [34]:
x = records | 'Writing to DB' >> relational_db.Write(
            source_config=source_config,
            table_config=table_config
        )

In [5]:
ib.show(x)

In [35]:
ib.show_graph(p)

# Below code for creating cloud dataflow instance

In [40]:

# Setting up the Apache Beam pipeline options.
options = PipelineOptions(flags=[])

# Sets the project to the default project in your current Google Cloud environment.
_, options.view_as(GoogleCloudOptions).project = google.auth.default()

# Sets the Google Cloud Region in which Cloud Dataflow runs.
options.view_as(GoogleCloudOptions).region = 'europe-west2'

dataflow_gcs_location = 'gs://third-campus-303308-bucket-jupyterlab/dataflow'

# Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary.
options.view_as(GoogleCloudOptions).staging_location = '%s/staging' % dataflow_gcs_location

# Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting to the sink.
options.view_as(GoogleCloudOptions).temp_location = '%s/temp' % dataflow_gcs_location

#options.view_as(SetupOptions).save_main_session = True

# Sets the requirements.txt file
#options.view_as(SetupOptions).requirements_file = './requirement.txt'

# Sets the setup.py file
options.view_as(SetupOptions).setup_file = './setup.py'


In [41]:
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

In [42]:
pipeline_result.wait_until_finish()

DataflowRuntimeException: Dataflow pipeline failed. State: FAILED, Error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/apache_beam/internal/pickler.py", line 286, in loads
    return dill.loads(s)
  File "/usr/local/lib/python3.7/site-packages/dill/_dill.py", line 275, in loads
    return load(file, ignore, **kwds)
  File "/usr/local/lib/python3.7/site-packages/dill/_dill.py", line 270, in load
    return Unpickler(file, ignore=ignore, **kwds).load()
  File "/usr/local/lib/python3.7/site-packages/dill/_dill.py", line 472, in load
    obj = StockUnpickler.load(self)
  File "/usr/local/lib/python3.7/site-packages/dill/_dill.py", line 462, in find_class
    return StockUnpickler.find_class(self, module, name)
ModuleNotFoundError: No module named 'sqlalchemy.cimmutabledict'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/dataflow_worker/batchworker.py

In [36]:
pip freeze

alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.7
anaconda-project==0.8.3
apache-beam==2.29.0
asn1crypto==1.4.0
astroid==2.3.1
astropy==3.2.1
atomicwrites==1.3.0
attrs==19.2.0
autosemver==0.5.5
avro-python3==1.9.2.1
Babel==2.7.0
backcall==0.1.0
backports.functools-lru-cache==1.5
backports.os==0.1.1
backports.shutil-get-terminal-size==1.0.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beam-nuggets==0.18.0
beamNuggetsDependencies==0.1
beautifulsoup4==4.8.0
bitarray==1.0.1
bkcharts==0.2
bleach==3.1.0
blinker==1.4
bokeh==1.3.4
boto==2.49.0
Bottleneck==1.2.1
cachetools==4.2.2
certifi==2019.9.11
cffi==1.12.3
chardet==3.0.4
Click==7.0
cloudpickle==1.2.2
clyent==1.2.2
colorama==0.4.1
comtypes==1.1.7
conda==4.10.1
conda-build==3.18.9
conda-package-handling==1.6.0
conda-verify==3.4.2
configobj==5.0.6
contextlib2==0.6.0
crcmod==1.7
cryptography==2.7
cycler==0.10.0
Cython==0.29.13
cytoolz==0.10.0
dask==2.5.2
decorator==4.4.0
defusedxml==0.6.0
dill==0.3.1.1
distributed==2.5